In [1]:
import stageemi
import stageemi.dev.visu as dev
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

import stageemi.dev.decorator_map as dm
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
import os 
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl
import pandas as pd
from datetime import datetime
import glob 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics.cluster import contingency_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
zone_dpt=[['FRJ13'],['FRB05'],['FRH02'],['FRK24']] #['FRB05' Loir-et-Cher,"FRH02" Finistere,'FRJ13' Herault,'FRK24' Isere]
zone_dpt

[['FRJ13'], ['FRB05'], ['FRH02'], ['FRK24']]

In [3]:
# choose which zone to process
zone_to_process=zone_dpt[0]

In [4]:
# path to score files
path="./scores_advanced_labia/"
# path to figures
path_figures="./scores_advanced_labia_figures/"

In [5]:
# WWMF codes
file_CodesWWMF = '../utils/CodesWWMF.csv'
df = pd.read_csv(file_CodesWWMF,usecols = (0,1,2,3,6,7),sep=',')
code_WWMF    = df['Code WWMF'].to_numpy()

In [6]:
for izone, zone in enumerate(zone_to_process):
                   
    print('Currently processing zone: '+zone)

    files=sorted(glob.glob(path+"*"+zone+"*"))  
    #print(files)

    scores=pd.read_csv(files[0],sep=',',index_col=0)
#     print('Currently reading file: '+files[0])        

    for ifile,fname in enumerate(files[1:]):    
        df_scores=pd.read_csv(fname,sep=',',index_col=0)
#         print('Currently reading file: '+fname)

        scores=scores.append(df_scores)   

        wwmf_codes=list(np.unique(scores["WWMF"]))
        scores["WWMF to WME"]='default'
        scores["WWMF to W1"]='default'

        for iwwmf,wwmf in enumerate(wwmf_codes):    
            scores["WWMF to WME"].loc[scores["WWMF"]==wwmf]=df[df["Code WWMF"]==wwmf]["Code WME"].iloc[0]    
            scores["WWMF to W1"].loc[scores["WWMF"]==wwmf]=df[df["Code WWMF"]==wwmf]["Code W1"].iloc[0]

Currently processing zone: FRJ13


/Users/gablellouch/anaconda3/envs/preproc/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# 1 WME/COMPAS

## 1.1 Confusion matrix

In [7]:
true_wme=np.unique(scores["WWMF to WME"])
pred_wme=np.unique(scores["WME"])
l_wme=len(list(set().union(true_wme,pred_wme)))
labels_wme=list(set().union(true_wme,pred_wme))
matrix_wme=confusion_matrix(scores["WWMF to WME"].to_list(),scores["WME"].to_list())
# matrix_wme
div_wme=np.transpose(np.ones((l_wme,1))*matrix_wme.sum(axis=1)).astype(int)
matrix_wme_norm=np.nan_to_num(matrix_wme/div_wme)
# matrix_wme_norm (normalized upon true wwmf values)
result_wme= ConfusionMatrixDisplay(matrix_wme,display_labels=labels_wme)
result_wme_norm= ConfusionMatrixDisplay(matrix_wme_norm,display_labels=labels_wme)
print('Number of elements (WME confusion matrix): '+str(matrix_wme.sum())) 

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,2,1)
result_wme.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix COMPAS: "+zone+"\n",fontsize=20)
ax = fig.add_subplot(1,2,2)
result_wme_norm.plot(ax=ax,values_format=".02f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix COMPAS (norm): "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.close()

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,1,1)
result_wme.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix COMPAS: "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.savefig(path_figures+'confusion_matrix_COMPAS_'+zone+'.png',bbox_inches = 'tight')
plt.close()

Number of elements (WME confusion matrix): 4224


## 1.2 Bar plot

In [8]:
# display bar plot for errors

correct_wme=100*np.nan_to_num(np.diag(matrix_wme)/matrix_wme.sum(axis=1))
error_wme=100*np.nan_to_num((matrix_wme.sum(axis=1)-np.diag(matrix_wme))/matrix_wme.sum(axis=1))

w_wme=matrix_wme.sum(axis=1)/matrix_wme.sum()
success_wme=sum(np.multiply(correct_wme,w_wme))

fig = plt.figure(figsize=(10,10))    
ax = fig.add_subplot(1,1,1)
ind_wme = np.arange(len(error_wme))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence
p1 = plt.bar(ind_wme, correct_wme, width)
p2 = plt.bar(ind_wme, error_wme, width,bottom=correct_wme)
plt.axhline(y=90, color='r', linestyle='--')
plt.axhline(y=success_wme, color='g', linestyle='--')
plt.xlabel('True label',fontsize=20)
plt.ylabel('Success rate (%)',fontsize=20)
plt.title('Success rate wrt Temps Maj.='+'{0:.2f}'.format(success_wme)+' - COMPAS: '+' '+zone+"\n",fontsize=20)
plt.xticks(ind_wme, labels_wme,fontsize=15)
plt.yticks(np.arange(0, 110, 10),fontsize=15)
plt.ylim((0,110))
plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'),fontsize=15,loc='upper right')
plt.savefig(path_figures+'success_rate_COMPAS_'+zone+'.png',bbox_inches = 'tight') 
plt.close()

# 2. WME/COMPAS ASYM

## 2.1 Confusion matrix

In [9]:
true_wme=np.unique(scores["WWMF to WME"])
pred_wme=np.unique(scores["WME_asym"])
l_wme=len(list(set().union(true_wme,pred_wme)))
labels_wme=list(set().union(true_wme,pred_wme))
matrix_wme=confusion_matrix(scores["WWMF to WME"].to_list(),scores["WME_asym"].to_list())
# matrix_wme
div_wme=np.transpose(np.ones((l_wme,1))*matrix_wme.sum(axis=1)).astype(int)
matrix_wme_norm=np.nan_to_num(matrix_wme/div_wme)
# matrix_wme_norm (normalized upon true wwmf values)
result_wme= ConfusionMatrixDisplay(matrix_wme,display_labels=labels_wme)
result_wme_norm= ConfusionMatrixDisplay(matrix_wme_norm,display_labels=labels_wme)
print('Number of elements (WME asym confusion matrix): '+str(matrix_wme.sum())) 

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,2,1)
result_wme.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix COMPAS ASYM: "+zone+"\n",fontsize=20)
ax = fig.add_subplot(1,2,2)
result_wme_norm.plot(ax=ax,values_format=".02f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix COMPAS ASYM (norm): "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.close()

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,1,1)
result_wme.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix COMPAS ASYM: "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.savefig(path_figures+'confusion_matrix_COMPAS_ASYM_'+zone+'.png',bbox_inches = 'tight')
plt.close()

<ipython-input-9-119524e8b9af>:8: RuntimeWarning: invalid value encountered in true_divide
  matrix_wme_norm=np.nan_to_num(matrix_wme/div_wme)


Number of elements (WME asym confusion matrix): 4224


## 2.1 Bar plot

In [10]:
correct_wme=100*np.nan_to_num(np.diag(matrix_wme)/matrix_wme.sum(axis=1))
error_wme=100*np.nan_to_num((matrix_wme.sum(axis=1)-np.diag(matrix_wme))/matrix_wme.sum(axis=1))

w_wme=matrix_wme.sum(axis=1)/matrix_wme.sum()
success_wme=sum(np.multiply(correct_wme,w_wme))

fig = plt.figure(figsize=(10,10))    
ax = fig.add_subplot(1,1,1)
ind_wme = np.arange(len(error_wme))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence
p1 = plt.bar(ind_wme, correct_wme, width)
p2 = plt.bar(ind_wme, error_wme, width,bottom=correct_wme)
plt.axhline(y=90, color='r', linestyle='--')
plt.axhline(y=success_wme, color='g', linestyle='--')
plt.xlabel('True label',fontsize=20)
plt.ylabel('Success rate (%)',fontsize=20)
plt.title('Success rate wrt Temps Maj.='+'{0:.2f}'.format(success_wme)+' - COMPAS ASYM: '+' '+zone+"\n",fontsize=20)
plt.xticks(ind_wme, labels_wme,fontsize=15)
plt.yticks(np.arange(0, 110, 10),fontsize=15)
plt.ylim((0,110))
plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'),fontsize=15,loc='upper right')
plt.savefig(path_figures+'success_rate_COMPAS_ASYM_'+zone+'.png',bbox_inches = 'tight') 
plt.close()

<ipython-input-10-b1872e6dca51>:1: RuntimeWarning: invalid value encountered in true_divide
  correct_wme=100*np.nan_to_num(np.diag(matrix_wme)/matrix_wme.sum(axis=1))
<ipython-input-10-b1872e6dca51>:2: RuntimeWarning: invalid value encountered in true_divide
  error_wme=100*np.nan_to_num((matrix_wme.sum(axis=1)-np.diag(matrix_wme))/matrix_wme.sum(axis=1))


# 3 W1/AGAT

## 3.1 Confusion matrix

In [11]:
true_w1=list(np.unique(scores["WWMF to W1"]))
pred_w1=list(np.unique(scores["W1"]))
l_w1=len(list(set().union(true_w1,pred_w1)))
labels_w1=list(set().union(true_w1,pred_w1))
matrix_w1=confusion_matrix(scores["WWMF to W1"].to_list(),scores["W1"].to_list())
# matrix_w1
div_w1=np.transpose(np.ones((l_w1,1))*matrix_w1.sum(axis=1)).astype(int)
matrix_w1_norm=np.nan_to_num(matrix_w1/div_w1)
# matrix_w1_norm (normalized upon true wwmf values)
result_w1= ConfusionMatrixDisplay(matrix_w1,display_labels=labels_w1)
result_w1_norm= ConfusionMatrixDisplay(matrix_w1_norm,display_labels=labels_w1)
print('Number of elements (W1 confusion matrix): '+str(matrix_w1.sum())) 

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,2,1)
result_w1.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix AGAT: "+zone+"\n",fontsize=20)
ax = fig.add_subplot(1,2,2)
result_w1_norm.plot(ax=ax,values_format=".02f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix AGAT (norm): "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.close()

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,1,1)
result_w1.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix AGAT: "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.savefig(path_figures+'confusion_matrix_AGAT_'+zone+'.png',bbox_inches = 'tight')            
plt.close()

<ipython-input-11-c9a40b843cab>:8: RuntimeWarning: invalid value encountered in true_divide
  matrix_w1_norm=np.nan_to_num(matrix_w1/div_w1)


Number of elements (W1 confusion matrix): 4224


## 3.2 Bar plot

In [12]:
# display bar plot for errors
correct_w1=100*np.nan_to_num(np.diag(matrix_w1)/matrix_w1.sum(axis=1))
error_w1=100*np.nan_to_num((matrix_w1.sum(axis=1)-np.diag(matrix_w1))/matrix_w1.sum(axis=1))

w_w1=matrix_w1.sum(axis=1)/matrix_w1.sum()
success_w1=sum(np.multiply(correct_w1,w_w1))

fig = plt.figure(figsize=(10,10))    
ax = fig.add_subplot(1,1,1)
ind_w1 = np.arange(len(error_w1))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence
p1 = plt.bar(ind_w1, correct_w1, width)
p2 = plt.bar(ind_w1, error_w1, width,bottom=correct_w1)
plt.axhline(y=90, color='r', linestyle='--')
plt.axhline(y=success_w1, color='g', linestyle='--')
plt.xlabel('True label',fontsize=20)
plt.ylabel('Success rate (%)',fontsize=20)
plt.title('Success rate wrt Temps Maj.='+'{0:.2f}'.format(success_w1)+' - AGAT: '+' '+zone+"\n",fontsize=20)
plt.xticks(ind_w1, labels_w1,fontsize=15)
plt.yticks(np.arange(0, 110, 10),fontsize=15)
plt.ylim((0,110))
plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'),fontsize=15,loc='upper right')
plt.savefig(path_figures+'success_rate_AGAT_'+zone+'.png',bbox_inches = 'tight') 
plt.close()

<ipython-input-12-8226be5a767a>:2: RuntimeWarning: invalid value encountered in true_divide
  correct_w1=100*np.nan_to_num(np.diag(matrix_w1)/matrix_w1.sum(axis=1))
<ipython-input-12-8226be5a767a>:3: RuntimeWarning: invalid value encountered in true_divide
  error_w1=100*np.nan_to_num((matrix_w1.sum(axis=1)-np.diag(matrix_w1))/matrix_w1.sum(axis=1))


# 4 W1/AGAT ASYM

## 4.1 Confusion matrix

In [13]:
true_w1=list(np.unique(scores["WWMF to W1"]))
pred_w1=list(np.unique(scores["W1_asym"]))
l_w1=len(list(set().union(true_w1,pred_w1)))
labels_w1=list(set().union(true_w1,pred_w1))
matrix_w1=confusion_matrix(scores["WWMF to W1"].to_list(),scores["W1_asym"].to_list())
# matrix_w1
div_w1=np.transpose(np.ones((l_w1,1))*matrix_w1.sum(axis=1)).astype(int)
matrix_w1_norm=np.nan_to_num(matrix_w1/div_w1)
# matrix_w1_norm (normalized upon true wwmf values)
result_w1= ConfusionMatrixDisplay(matrix_w1,display_labels=labels_w1)
result_w1_norm= ConfusionMatrixDisplay(matrix_w1_norm,display_labels=labels_w1)
print('Number of elements (W1 confusion matrix): '+str(matrix_w1.sum())) 

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,2,1)
result_w1.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix AGAT ASYM: "+zone+"\n",fontsize=20)
ax = fig.add_subplot(1,2,2)
result_w1_norm.plot(ax=ax,values_format=".02f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix AGAT ASYM (norm): "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.close()

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1,1,1)
result_w1.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
plt.xlabel("Predicted label",fontsize=20)
plt.ylabel("True label",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Confusion matrix AGAT ASYM: "+zone+"\n",fontsize=20)
plt.tight_layout()
plt.savefig(path_figures+'confusion_matrix_AGAT_ASYM_'+zone+'.png',bbox_inches = 'tight')            
plt.close()

<ipython-input-13-89eccd7d8452>:8: RuntimeWarning: invalid value encountered in true_divide
  matrix_w1_norm=np.nan_to_num(matrix_w1/div_w1)


Number of elements (W1 confusion matrix): 4224


## 4.2 Bar plot

In [14]:
# display bar plot for errors
correct_w1=100*np.nan_to_num(np.diag(matrix_w1)/matrix_w1.sum(axis=1))
error_w1=100*np.nan_to_num((matrix_w1.sum(axis=1)-np.diag(matrix_w1))/matrix_w1.sum(axis=1))

w_w1=matrix_w1.sum(axis=1)/matrix_w1.sum()
success_w1=sum(np.multiply(correct_w1,w_w1))

fig = plt.figure(figsize=(10,10))    
ax = fig.add_subplot(1,1,1)
ind_w1 = np.arange(len(error_w1))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence
p1 = plt.bar(ind_w1, correct_w1, width)
p2 = plt.bar(ind_w1, error_w1, width,bottom=correct_w1)
plt.axhline(y=90, color='r', linestyle='--')
plt.axhline(y=success_w1, color='g', linestyle='--')
plt.xlabel('True label',fontsize=20)
plt.ylabel('Success rate (%)',fontsize=20)
plt.title('Success rate wrt Temps Maj.='+'{0:.2f}'.format(success_w1)+' - AGAT ASYM: '+' '+zone+"\n",fontsize=20)
plt.xticks(ind_w1, labels_w1,fontsize=15)
plt.yticks(np.arange(0, 110, 10),fontsize=15)
plt.ylim((0,110))
plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'),fontsize=15,loc='upper right')
plt.savefig(path_figures+'success_rate_AGAT_ASYM_'+zone+'.png',bbox_inches = 'tight') 
plt.close()

<ipython-input-14-ba97409f4ee1>:2: RuntimeWarning: invalid value encountered in true_divide
  correct_w1=100*np.nan_to_num(np.diag(matrix_w1)/matrix_w1.sum(axis=1))
<ipython-input-14-ba97409f4ee1>:3: RuntimeWarning: invalid value encountered in true_divide
  error_w1=100*np.nan_to_num((matrix_w1.sum(axis=1)-np.diag(matrix_w1))/matrix_w1.sum(axis=1))
